In [86]:
import random

from deap import base
from deap import creator
from deap import tools
import numpy 
from deap import algorithms
from itertools import combinations

from collections import defaultdict
from random import randrange, shuffle
import importlib


import config
importlib.reload(config)
from config import NUM_EXAMS, NUM_SLOTS, NUM_STUDENTS

print("config:", NUM_EXAMS, NUM_SLOTS, NUM_STUDENTS)
 
EXAMS = ['exam_%s' %s for s in range(NUM_EXAMS)]


config: 73 31 100000


In [87]:

def load_data(data_file):
    students_exams = defaultdict()
    
    with open(data_file, "r") as f:
        lines = f.readlines()
        
        for line in lines:            
            student, exams = line.strip().split("\t")
            students_exams[student] = exams.split(",")
    
    return students_exams


def generate_combinations( L ): 
    ret = [ c for c in combinations(L, 2) ] 
    return ret


def examsToSlots(exams, numSlots):
    """
    map exams -> slotsToExams
    exam1 --> slot45
    exam2 --> slot2
    exam3 --> slot13
    .....
    """

    ex2Slots = {}
    for item in exams:      
        ex2Slots[item] = randrange(numSlots)
    return ex2Slots
 
     
def generateExams(exams_list = EXAMS): 
    """
    generate population for deap
    create a permutation of [0,1,2,...,NUM_SLOTS]
    and assign exam --> permutation[] 
    """
    xperm = [ 'slot_%s' % i for i in range(NUM_SLOTS)]
    shuffle( xperm  )
 
    i = 0
    for item in range(NUM_EXAMS): 
        i =  (i +1)
        i = i % NUM_SLOTS
        yield exams_list[item],  xperm[i] #randrange(NUM_SLOTS)
        
        
pop = generateExams(EXAMS)
for p in pop: print(p)

('exam_0', 'slot_21')
('exam_1', 'slot_16')
('exam_2', 'slot_12')
('exam_3', 'slot_18')
('exam_4', 'slot_3')
('exam_5', 'slot_9')
('exam_6', 'slot_8')
('exam_7', 'slot_0')
('exam_8', 'slot_13')
('exam_9', 'slot_10')
('exam_10', 'slot_6')
('exam_11', 'slot_17')
('exam_12', 'slot_2')
('exam_13', 'slot_5')
('exam_14', 'slot_29')
('exam_15', 'slot_26')
('exam_16', 'slot_4')
('exam_17', 'slot_23')
('exam_18', 'slot_25')
('exam_19', 'slot_20')
('exam_20', 'slot_22')
('exam_21', 'slot_28')
('exam_22', 'slot_30')
('exam_23', 'slot_1')
('exam_24', 'slot_24')
('exam_25', 'slot_19')
('exam_26', 'slot_11')
('exam_27', 'slot_14')
('exam_28', 'slot_7')
('exam_29', 'slot_15')
('exam_30', 'slot_27')
('exam_31', 'slot_21')
('exam_32', 'slot_16')
('exam_33', 'slot_12')
('exam_34', 'slot_18')
('exam_35', 'slot_3')
('exam_36', 'slot_9')
('exam_37', 'slot_8')
('exam_38', 'slot_0')
('exam_39', 'slot_13')
('exam_40', 'slot_10')
('exam_41', 'slot_6')
('exam_42', 'slot_17')
('exam_43', 'slot_2')
('exam_44', 's

In [88]:
SLOTS = [ "slot_%s" %s for s in range(NUM_SLOTS)]
STUDENTS_EXAMS_LOOKUP = load_data("students_exams.txt")

In [89]:
OVERLAPS = defaultdict( defaultdict )
with open("overlaps.txt", "r") as f:
    for line in f.readlines():
        ex1, ex2, count = line.strip().split("\t")        
        OVERLAPS[ex1][ex2] = int(count)
        

        
        
def calc_total_cost(slots):
    cost = 0 
    for slot in slots:
        combs =  generate_combinations( slots[slot] )
        
        for c in combs:
            ex1, ex2 = c
            if ex1>ex2: ex1, ex2 = ex2, ex1
            cost+= OVERLAPS[ex1][ex2]

    return cost




"""
    crossover two items
    split items in half and merge the first part of A with the 2nd part of B and the first part of B with the second part of A
"""
def cxSet(item1, item2):

    cxPt = NUM_EXAMS/2 #randrange( NUM_EXAMS )   

    newItem1 = creator.Individual()
    newItem2 = creator.Individual() 
    for i in range(NUM_EXAMS):
        entry = EXAMS[i]
        if i<cxPt:
            newItem1[ entry ] = item1[entry]
            newItem2[ entry ] = item2[entry]
        else:
            newItem1[ entry ] = item2[entry]
            newItem2[ entry ] = item1[entry]
    return newItem1, newItem2

"""
    mutate an individual
    re-assign a random exam to a random slot
"""
def mutSet(individual):
   
    if random.random() < 0.6:
        rand_exam = EXAMS[randrange(NUM_EXAMS)]
        srcSlot  = randrange(NUM_SLOTS)
        destSlot = randrange(NUM_SLOTS)
        individual[rand_exam] = 'slot_%s' % destSlot 
        
    return individual,

In [90]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMax)


toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", random.randint, 0, 1)
# Structure initializers

toolbox.register("individual", tools.initIterate, creator.Individual,  generateExams) 
toolbox.register("population", tools.initRepeat,  list  , toolbox.individual)

toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selBest)


def evalSolution(individual):

    slotsToExams = defaultdict(list)
    for slot in range(NUM_SLOTS): 
        slotsToExams[slot]  = []
    
    for exam in individual: 
        slot = individual[exam]
        slotsToExams[slot]+= [exam]
    cost =  calc_total_cost(slotsToExams)    
    return cost,



toolbox.register("evaluate", evalSolution)



In [91]:
def main():
    
    NGEN = 460
    MU = 300
    LAMBDA = 300
    CXPB = 0.75
    MUTPB = 0.25
    
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, log = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    return pop,log

In [92]:
pop, log=main()

gen	nevals	avg  	std	min  	max  
0  	300   	17059	0  	17059	17059
1  	300   	17053.8	21.9662	16853	17059
2  	300   	17048.3	30.1832	16853	17059
3  	300   	17044  	36.1869	16853	17059
4  	300   	17034.7	48.5282	16731	17059
5  	300   	17025.8	52.6937	16731	17059
6  	300   	17015.8	56.1784	16731	17059
7  	300   	17002.1	59.3467	16731	17059
8  	300   	16981.2	64.1082	16725	17056
9  	300   	16964.8	60.9101	16725	17034
10 	300   	16951.4	59.0807	16725	17018
11 	300   	16939.1	57.364 	16725	17007
12 	300   	16918.5	55.1913	16725	16986
13 	300   	16903.8	50.8698	16725	16969
14 	300   	16889.9	45.8913	16725	16952
15 	300   	16875.3	42.97  	16697	16934
16 	300   	16864.7	37.7724	16697	16916
17 	300   	16854.3	35     	16697	16899
18 	300   	16842.4	33.4685	16697	16880
19 	300   	16835.6	31.8547	16697	16867
20 	300   	16828.3	31.0384	16697	16862
21 	300   	16820.5	28.2533	16697	16852
22 	300   	16814.8	27.8427	16697	16842
23 	300   	16807.6	27.5507	16697	16839
24 	300   	16800.6	25.7647	16697	1682

In [93]:
print( pop[-1])

{'exam_56': 'slot_24', 'exam_6': 'slot_21', 'exam_58': 'slot_16', 'exam_51': 'slot_14', 'exam_47': 'slot_9', 'exam_38': 'slot_3', 'exam_19': 'slot_23', 'exam_62': 'slot_12', 'exam_18': 'slot_19', 'exam_2': 'slot_18', 'exam_1': 'slot_5', 'exam_33': 'slot_13', 'exam_37': 'slot_11', 'exam_55': 'slot_1', 'exam_50': 'slot_15', 'exam_59': 'slot_22', 'exam_13': 'slot_3', 'exam_16': 'slot_17', 'exam_48': 'slot_25', 'exam_12': 'slot_22', 'exam_34': 'slot_15', 'exam_64': 'slot_14', 'exam_20': 'slot_3', 'exam_3': 'slot_18', 'exam_65': 'slot_8', 'exam_15': 'slot_10', 'exam_44': 'slot_13', 'exam_41': 'slot_5', 'exam_67': 'slot_6', 'exam_10': 'slot_4', 'exam_32': 'slot_12', 'exam_71': 'slot_7', 'exam_53': 'slot_29', 'exam_39': 'slot_19', 'exam_29': 'slot_10', 'exam_68': 'slot_21', 'exam_36': 'slot_17', 'exam_7': 'slot_20', 'exam_52': 'slot_29', 'exam_45': 'slot_30', 'exam_5': 'slot_17', 'exam_60': 'slot_28', 'exam_22': 'slot_0', 'exam_0': 'slot_16', 'exam_49': 'slot_16', 'exam_8': 'slot_2', 'exam_27

In [94]:
solution = pop[-1]

In [95]:
len( solution.keys() )

73

In [96]:
len( set( solution.values() ) )

31

In [97]:
print( evalSolution(solution))

(15702,)


In [100]:
import math
total = 0

students_overlaps = {}

for idx, student in enumerate(STUDENTS_EXAMS_LOOKUP):
    exams = STUDENTS_EXAMS_LOOKUP[student]
    #print(idx, student, exams )
    
    slots = [ solution[exam] for exam in exams]
    
    scount = defaultdict(int)
    for s in slots:
        scount[s]+=1
    
    st_cnt = 0
    for s in scount:
        if scount[s]>1:
            #print( "count=>", scount[s], s) 
            st_cnt+= math.factorial( scount[s]  )
            
    total+=st_cnt
    
    students_overlaps[student] = st_cnt/2
        
        
            
    #print(idx, student, slots )
    
        
print("total is :",  total/2)

total is : 15702.0


In [109]:
x = list(students_overlaps.items())

In [128]:
import numpy as np
zeros = np.asarray([item for item in x if item[1]==0])
ones = np.asarray([item for item in x if item[1]==1])
twos = np.asarray([item for item in x if item[1]==2])
threes = np.asarray([item for item in x if item[1]==3])
fours = np.asarray([item for item in x if item[1]==4])

In [129]:
zeros.shape, ones.shape, twos.shape

((85034, 2), (14359, 2), (483, 2))

In [130]:
zeros.shape[0]+ones.shape[0]+twos.shape[0] + threes.shape[0] + fours.shape[0]

100000